In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

RAW_DATA_FOLDER = os.getenv('RAW_DATA_FOLDER', '/data/raw')
CLEAN_DATA_FOLDER = os.getenv('CLEAN_DATA_FOLDER', '/data/clean')

In [3]:
import polars as pl

cryptonews_df = pl.read_parquet(f"{RAW_DATA_FOLDER}/cryptopanic_news.parquet")
cryptonews_df.head(5)

id,title,description,sourceId,sourceDomain,sourceUrl,newsDatetime,url,negative,positive,important,liked,disliked,lol,toxic,saved,comments,currencies
i64,str,str,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
873,"""ApeCoin Community Votes to Sta…","""NULL""",2,"""u.today""","""NULL""","""2022-06-09 06:57:15""","""https://cryptopanic.com/news/1…",0,0,0,0,0,0,0,0,0,"""ETH,APE"""
875,"""Terraform Labs Co-Founder Deni…","""NULL""",2,"""u.today""","""NULL""","""2022-06-09 06:03:33""","""https://cryptopanic.com/news/1…",0,0,0,1,0,0,0,0,0,"""NULL"""
876,"""Anchor dev claims he warned Do…","""NULL""",7,"""cointelegraph.com""","""NULL""","""2022-06-09 05:39:22""","""https://cryptopanic.com/news/1…",1,0,1,0,0,0,0,0,0,"""LUNC"""
877,"""With 12 Key Measures of Prices…","""NULL""",11,"""newsbtc.com""","""NULL""","""2022-06-09 05:32:52""","""https://cryptopanic.com/news/1…",0,0,0,0,0,0,0,0,0,"""XCN"""
878,"""'Father' of the Metaverse Neal…","""NULL""",7,"""cointelegraph.com""","""NULL""","""2022-06-09 05:28:31""","""https://cryptopanic.com/news/1…",0,1,0,2,0,0,0,0,0,"""NULL"""


In [4]:
# we don't want to include community votes for sentiment analysis
# title + description should suffice
# added cryptopanic prefix to avoid confusion with other news sources 
# although we only are using cryptopanic for now
cryptonews_core_df = cryptonews_df.select([
    pl.col("id").alias("cryptopanic_id"),
    pl.col("title"),
    pl.col("description"),
    pl.col("sourceDomain").alias("source_domain"),
    # pl.col("sourceUrl").alias("source_url"), # og source URL, not needed for now
    pl.col("newsDatetime").alias("published_at"),
    pl.col("url").alias("cryptopanic_url"),
    pl.col("currencies")
])

cryptonews_core_df.head(5)

cryptopanic_id,title,description,source_domain,published_at,cryptopanic_url,currencies
i64,str,str,str,str,str,str
873,"""ApeCoin Community Votes to Sta…","""NULL""","""u.today""","""2022-06-09 06:57:15""","""https://cryptopanic.com/news/1…","""ETH,APE"""
875,"""Terraform Labs Co-Founder Deni…","""NULL""","""u.today""","""2022-06-09 06:03:33""","""https://cryptopanic.com/news/1…","""NULL"""
876,"""Anchor dev claims he warned Do…","""NULL""","""cointelegraph.com""","""2022-06-09 05:39:22""","""https://cryptopanic.com/news/1…","""LUNC"""
877,"""With 12 Key Measures of Prices…","""NULL""","""newsbtc.com""","""2022-06-09 05:32:52""","""https://cryptopanic.com/news/1…","""XCN"""
878,"""'Father' of the Metaverse Neal…","""NULL""","""cointelegraph.com""","""2022-06-09 05:28:31""","""https://cryptopanic.com/news/1…","""NULL"""


In [5]:
# some columns have "NULL" as string, we will replace them with None
# datetime is in string format, we will convert it to datetime
# also, we will cast cryptopanic_id to Int64 for consistency

NULL_VALUES = ["NULL", "null", "", "-"]


cryptonews_core_df = cryptonews_core_df.with_columns(
    pl.col("cryptopanic_id").cast(pl.Int64),
    pl.col("title").str.strip_chars().replace(NULL_VALUES, None),
    pl.col("description").str.strip_chars().replace(NULL_VALUES, None),
    pl.col("source_domain").str.strip_chars().replace(NULL_VALUES, None),
    pl.col("cryptopanic_url").str.strip_chars().replace(NULL_VALUES, None),
    pl.col("published_at").str.to_datetime(format="%Y-%m-%d %H:%M:%S").cast(pl.Datetime),
    pl.col("currencies").str.strip_chars().replace(NULL_VALUES, None)
).filter(
    pl.col("title").is_not_null()
)

cryptonews_core_df.head(5)

cryptopanic_id,title,description,source_domain,published_at,cryptopanic_url,currencies
i64,str,str,str,datetime[μs],str,str
873,"""ApeCoin Community Votes to Sta…",null,"""u.today""",2022-06-09 06:57:15,"""https://cryptopanic.com/news/1…","""ETH,APE"""
875,"""Terraform Labs Co-Founder Deni…",null,"""u.today""",2022-06-09 06:03:33,"""https://cryptopanic.com/news/1…",null
876,"""Anchor dev claims he warned Do…",null,"""cointelegraph.com""",2022-06-09 05:39:22,"""https://cryptopanic.com/news/1…","""LUNC"""
877,"""With 12 Key Measures of Prices…",null,"""newsbtc.com""",2022-06-09 05:32:52,"""https://cryptopanic.com/news/1…","""XCN"""
878,"""'Father' of the Metaverse Neal…",null,"""cointelegraph.com""",2022-06-09 05:28:31,"""https://cryptopanic.com/news/1…",null


In [6]:
cryptonews_core_df.describe()

statistic,cryptopanic_id,title,description,source_domain,published_at,cryptopanic_url,currencies
str,f64,str,str,str,str,str,str
"""count""",216455.0,"""216455""","""23301""","""216455""","""216455""","""216455""","""215496"""
"""null_count""",0.0,"""0""","""193154""","""0""","""0""","""0""","""959"""
"""mean""",117547.471525,null,null,null,"""2023-04-04 09:15:58.055753""",null,null
"""std""",62904.719266,null,null,null,null,null,null
"""min""",873.0,"""!!️!!️#NOLEARMY ATTENTION !!️!…","""""A good day for the crypto sec…","""99bitcoins.com""","""2017-09-23 10:00:42""","""https://cryptopanic.com/news/1…","""0x0"""
"""25%""",63446.0,null,null,null,"""2022-05-05 18:41:09""",null,null
"""50%""",117671.0,null,null,null,"""2023-04-20 17:54:23""",null,null
"""75%""",171941.0,null,null,null,"""2024-03-22 17:31:04""",null,null
"""max""",226199.0,"""🫶""","""🚀 Learn Technical Analysis wit…","""zycrypto.com""","""2025-05-24 07:38:26""","""https://cryptopanic.com/news/u…","""ZRX,NOT"""


Too many rows without description, we'll combine title and description for sentiment analysis

We'll remove the rows without currencies

In [7]:
def check_duplicates(df: pl.DataFrame, subset: list, print_head: bool = False) -> None:
    """
    Check for duplicates in the DataFrame based on the specified subset of columns.
    """
    duplicates = df.group_by(subset).agg(pl.len().alias("count")).filter(pl.col("count") > 1)
    if duplicates.shape[0] > 0:
        print(f"Found {duplicates.shape[0]} duplicate entries based on {subset}:")
        if print_head:
            for col in subset:
                print(f"Sample duplicate {col}:")
                print(df.filter(pl.col(col).is_in(duplicates[col].implode())).head(5))
    else:
        print("No duplicates found.")


check_duplicates(cryptonews_core_df, ["cryptopanic_url"])
check_duplicates(cryptonews_core_df, ["title", "description"])

Found 2 duplicate entries based on ['cryptopanic_url']:
Found 14 duplicate entries based on ['title', 'description']:


In [8]:
# delete rows with no currencies and no description
cryptonews_clean_df = cryptonews_core_df.filter(
    pl.col("currencies").is_not_null() &
    pl.col("description").is_not_null()
)

# remove duplicates based on cryptopanic_url unique identifier
# remove duplicates based on title and description pair
cryptonews_clean_df = (cryptonews_clean_df
    .unique(subset=["cryptopanic_url"], keep="first")
    .unique(subset=["title", "description"], keep="first")
)

cryptonews_clean_df.describe()

statistic,cryptopanic_id,title,description,source_domain,published_at,cryptopanic_url,currencies
str,f64,str,str,str,str,str,str
"""count""",22897.0,"""22897""","""22897""","""22897""","""22897""","""22897""","""22897"""
"""null_count""",0.0,"""0""","""0""","""0""","""0""","""0""","""0"""
"""mean""",93707.539459,null,null,null,"""2022-10-09 17:25:27.520811""",null,null
"""std""",49059.433347,null,null,null,null,null,null
"""min""",911.0,"""""Bitcoin Influencer"" Peter Sch…","""""A good day for the crypto sec…","""99bitcoins.com""","""2021-01-01 09:34:00""","""https://cryptopanic.com/news/1…","""1INCH"""
"""25%""",78317.0,null,null,null,"""2021-11-24 16:07:19""",null,null
"""50%""",84630.0,null,null,null,"""2022-05-12 11:27:00""",null,null
"""75%""",98853.0,null,null,null,"""2023-10-21 15:32:00""",null,null
"""max""",226185.0,"""‪GoDaddy taps Ethereum Name Se…","""🚀 Learn Technical Analysis wit…","""zycrypto.com""","""2025-05-23 19:55:52""","""https://cryptopanic.com/news/u…","""ZRX"""


In [9]:
cryptonews_clean_df.write_parquet(f"{CLEAN_DATA_FOLDER}/cryptopanic_news_clean.parquet")

In [10]:
# richer report for the cleaned data
from ydata_profiling import ProfileReport

report = ProfileReport(
    cryptonews_clean_df.to_pandas(),
    title="Cryptopanic News Data Profiling Report",
    explorative=True,
    minimal=True,
    progress_bar=False
)

report.to_notebook_iframe()

100%|██████████| 7/7 [00:01<00:00,  5.38it/s]
